# COMP90051 Project 2
**Name:** *enter your name here*

**Student ID:** *your id here*

In [28]:
# You can add additional imports here
import numpy as np
from matplotlib import pyplot as plt
import os
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict
# Additional imports
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from numpy import mean
from numpy import std


## 0. Loading the dataset

In [3]:
# do not edit this cell
# load the data files (download from the LMS)
embedded_images = np.load('images.npy')
labels = np.load('labels.npy')

# split into pool & testing
X_pool, X_test, y_pool, y_test = train_test_split(embedded_images, labels, 
                                                  test_size=0.5, random_state=1234, shuffle=True)

# sample a seed set
np.random.seed(1234)
label2id = defaultdict(list)
for i, label in enumerate(y_pool):
    label2id[label].append(i)
seed_set = []
for label, ids in label2id.items():
    seed_set.extend(np.random.choice(ids, size=10, replace=False))

In [39]:
#X_pool.shape #(9640, 1000)
#y_pool.shape #(9640,)
#len(label2id) #30
#label2id #30
X_pool[seed_set].shape


(300, 1000)

## 1. Applying logistic regression

In [20]:
def train_logistic_regression(X, y, **args):
    """
    Train a logistic regression model on dataset (X, y) and return trained model.
    X: matrix of real values, size n x d
    y: vector of string labels, size n
    args: optional arguments e.g., for hyper-parameters
    """
    # your code here
    # run logistic regression model
    log_reg = LogisticRegression(multi_class='multinomial',  solver='lbfgs')
    model = log_reg.fit(X, y)
    return model


In [44]:
def evaluate_logistic_regression_accuracy(Xt, yt, model):
    """
    Apply logistic regression prediction on dataset Xt and evaluate accuracy against yt,
    returing the accuracy results as a scalar.
    Xt: matrix of real values, size m x d
    yt: vector of string labels, size m
    """
    # your code here
    # test prediction
    y_pred = model.predict(Xt)
    # print("Accuracy: {}".format(accuracy_score(yt, y_pred)))
    # print("Precision: {}".format(precision_score(yt, y_pred)))
    # print("Recall: {}".format(recall_score(yt, y_pred)))
    
    # evaluate the model and collect the scores
    # define the model evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    n_scores = cross_val_score(model, Xt, yt, scoring='accuracy', cv=cv, n_jobs=-1)
    # report the model performance
    print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
  

In [42]:
# your code here for training, evaluating & plotting results

#model 1 Seed Set
model1 = train_logistic_regression(X_pool[seed_set],seed_set)

#model 2 Max Pool 
#model2 = train_logistic_regression(X_pool,y_pool)

C:\Users\Waqar\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [45]:
evaluate_logistic_regression_accuracy(X_test,y_test,model1)

Mean Accuracy: 0.561 (0.016)


## 2. Active learning framework with Random selection

In [56]:
def random_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a model (not used)
    returns a vector of scores of length n. Each entry reflects the priority 
    of the corresponding instance. Higher means better.
    """
    # fill in

    # Creating a score array using random numbers of length of X 
    scores = np.random.randint(1,len(X),len(X))
    return scores
    

In [66]:
    #test scoring function
    #scores = random_select(X_pool,model)
    print(len(scores))
    scores

9640


array([ 516, 7830, 6294, ...,  466, 1933,  865])

In [159]:
#  start= X_pool[seed_set]
#  print(test.shape)
#  print(X_pool.shape)
#  len(test)
#  a1_rows = X_pool.view([('', X_pool.dtype)] * X_pool.shape[1])
#  a2_rows = start.view([('', start.dtype)] * start.shape[1])
#  c = np.setdiff1d(a1_rows, a2_rows).view(X_pool.dtype).reshape(-1, X_pool.shape[1])
#  print(c.shape)
 
 
 


(300, 1000)
(9640, 1000)
(9340, 1000)


array([[-7.697327  , -5.2156897 , -1.859092  , ..., -3.6263976 ,
        -2.8793776 ,  4.6810117 ],
       [-6.775587  , -5.8626003 , -3.096824  , ..., -5.0418954 ,
        -3.3128798 ,  3.6669626 ],
       [-6.675805  , -5.1676207 , -2.9839501 , ..., -4.72577   ,
        -2.6260886 ,  7.5567617 ],
       ...,
       [ 0.41712472, -1.8523746 , -1.018431  , ..., -0.6250134 ,
         0.62452245, -0.8468784 ],
       [ 0.4389726 , -2.1272082 , -0.17403641, ..., -1.654047  ,
         0.71137625, -0.97278035],
       [ 0.7555078 , -1.6039836 , -1.058054  , ..., -1.7256362 ,
        -0.32721663, -1.4546849 ]], dtype=float32)

In [ ]:
def pool_based_active_learning(X_pool, y_pool, seed_ids,
                               train_func, select_func,
                               max_size, batch_size, **args):
    """
    Perform an active learning simulation, which starts by training on a seed set,
    then iteratively applies the selection function to rank instances in the pool,
    selects the top few instances which are included into the training set and the
    process repeats. 
        X_pool: matrix of n x d
        y_pool: vector of string labels, size n
        seed_ids: initial labelled set set, as a list of indices [0..n-1] into pool
        train_func: function which given (X, y, optional args) returns a trained model
        select_func: function which given (X, optional args) returns a sequence of scores
        max_size: stopping condition for active learning, when labelled data reaches given size
        batch_size: number of instances to be labelled in each iteration
        args: optional arguments passed to training and selection function
    returns the sequence of trained models 
    """
  
    # fill in
    # 1: U = pool of unlabelled instances, {x} 
    # 2: L = set of initial labelled instances, {hx,yi} 
    # 3: b = number of instances to label in each step 
    # 4: for t = 1,2,...,T do 
    # 5:  θ(t) = train(L) 
    # 6:  score all instances in pool, r = select(U) 
    # 7:  for all j ∈ argmax(b,r) do 
    # 8:      reveal label yj 
    # 9:      add hxj,yji to L 
    # 10:     remove xj from U 
    # 11:    end for 
    # 12: end for 
    # 13: return {θ(t)}T t=1



    

In [ ]:
batch = 60
max_size = 3000
models_random = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                    train_logistic_regression, random_select, 
                                    max_size, batch)

In [ ]:
## your code here for evaluation of accuracy and plotting of results

## 3. Uncertainty sampling

In [ ]:
def logistic_regression_entropy_select(X, model, **args):
    """
    Given an unlabelled dataset X, a matrix of n x d, and a discriminative model 
    P(y|x), returns a vector of n entropy values.
    """
    # fill in
    pass

In [ ]:
models_us = pool_based_active_learning(X_pool, y_pool, seed_set, 
                                 train_logistic_regression, 
                                 logistic_regression_entropy_select, 
                                 max_size, batch)

In [ ]:
## your code here for evaluation of accuracy and plotting of results

## 4. Query by committee

In [ ]:
# don't forget to provide function descriptive comments, like those provided in templates above

def query_by_committee_vote_entropy(X, model, **args):
    pass

def query_by_committee_soft_vote_entropy(X, model, **args):
    pass

def query_by_committee_KL(X, model, **args):
    pass

In [ ]:
def train_committee(X, y, **args):
    pass

In [ ]:
## your code here for training, evaluation, and plotting code

## 5. Hierarchical sampling

In [1]:
## your code